 # Run streamlit app in Jupiter notebook

As you know, ngrok is a reverse proxy tool that allows you to securely expose a local web server to the internet. Essentially, it provides a public URL that you can use to access your local web server from anywhere in the world. It's a good choice when you want to make it easier to test and share web applications that are running on your local machine.
Instead, Streamlit is an open-source Python library for building web applications for machine learning and data science.

This very simple example try to put them together.

## Cleanup

Because we need some stufk like streamlit, ngrok, pyngrok, protobuf we had to be sure that the dowload folder is empty.

In [ ]:
import os

path = "/kaggle/working/"
for file_name in os.listdir(path):
    file = path + file_name
    if os.path.isfile(file):
        print('Deleting file:', file)
        os.remove(file)

Let's to unzip our pipeline

In [ ]:
!unzip pipeline/sentiment_pipeline.zip

## Install stuff

So we'll install the streamlit, ngrok, pyngrok, protobuf, and validators packages using the pip install command. 
The --no-dependencies and --quiet options are used to prevent the installation of unnecessary dependencies and to prevent output from being displayed on the console, respectively.

In [ ]:
!pip install --no-dependencies --quiet streamlit

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

In [ ]:
!unzip -o ngrok-stable-linux-amd64.zip

In [ ]:
!pip install --quiet pyngrok

In [ ]:
!pip install --no-dependencies --quiet protobuf==3.20.*   #==4.21.12

In [ ]:
!pip install --no-dependencies --quiet validators

In [ ]:
# from kaggle_secrets import UserSecretsClient

# secret_label = "myToken"
# secret_value = UserSecretsClient().get_secret(secret_label)

The ngrok tool is authenticated using an auth token. This authentication allows the ngrok tool to access the internet and to provide secure tunnels to the web application.
<b>I've tried to use the Kaggle Secrets add-on; unfortunately he doesn't work properly.</b>


In [ ]:
!ngrok authtoken "<PUT_YOUR_TOKEN_HERE>" 

The script then uses the streamlit library to create a simple web application for sentiment analysis (useful if you want to predict the sentiment of an input text).
After the model is used, the result is displayed as a bar plot using the seaborn and matplotlib packages.

In [ ]:
%%writefile my_app.py
import streamlit as st
from joblib import load
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

@st.cache(allow_output_mutation=True)
def load_model():
    print('loading model')
    pipeline = load("pipeline/sentiment_pipeline.joblib")
    return pipeline


model = load_model()

st.title('Sentiment analysis')

review = st.text_input("What do you think about?", "This movie sucks! My whole family hates it!")
sentiment = model.predict_proba([review])

#st.write(sentiment)
fig, ax = plt.subplots(figsize=(5, 5))
sns.barplot(x=['Negative', 'Positive'], y=sentiment[0], ax=ax)

fig.savefig("figure_name.png")
image = Image.open('figure_name.png')
st.image(image)

The public URL of the sentiment analysis web application is obtained using the ngrok tool and printed to the console. The ngrok.connect() function is used to establish a connection to the web application, which is running on port 80.
You will see something like: "http://0fb6-34-81-188-91.ngrok.io" -> "http://localhost:80".

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(port = '80')
print(f"Please click on the text below {public_url}")

Finally, the script runs the sentiment analysis web application using the streamlit run command.
The "> /dev/null" option is used to redirect the output of the application to /dev/null, so it is not displayed on the console.

Please note that you have to stop the following statement after 

In [ ]:
!streamlit run --server.port 80 my_app.py > /dev/null